In [1]:
from keras.preprocessing.image import load_img, img_to_array
target_image_path = 'pic.jpeg'
style_reference_image_path = 'OIP2.jfif'
width, height = load_img(target_image_path).size
img_height = 400
img_width = int(width * img_height / height)

Using TensorFlow backend.


In [2]:
import numpy as np
from keras.applications import vgg19
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_height, img_width))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [3]:
def deprocess_image(x):
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [4]:
from keras import backend as K
target_image = K.constant(preprocess_image(target_image_path))
style_reference_image = K.constant(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_height, img_width, 3))
input_tensor = K.concatenate([target_image,style_reference_image, combination_image], axis=0)
model = vgg19.VGG19(input_tensor=input_tensor,weights='imagenet',include_top=False)
print('Model loaded.')

Model loaded.


In [5]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [6]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [7]:
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_height * img_width
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [8]:
def total_variation_loss(x):
    a = K.square(
    x[:, :img_height - 1, :img_width - 1, :] -
    x[:, 1:, :img_width - 1, :])
    b = K.square(
    x[:, :img_height - 1, :img_width - 1, :] -
    x[:, :img_height - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [9]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
content_layer = 'block5_conv2'
style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
total_variation_weight = 1e-4
style_weight = 1.
content_weight = 0.50

In [10]:
loss = K.variable(0.)
layer_features = outputs_dict[content_layer]
target_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(target_image_features,combination_features)

In [11]:
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(style_layers)) * sl


In [12]:
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [13]:
grads = K.gradients(loss, combination_image)[0]
fetch_loss_and_grads = K.function([combination_image], [loss, grads])

In [14]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values    
    
evaluator = Evaluator()

In [15]:
from scipy.optimize import fmin_l_bfgs_b
import imageio
import time
result_prefix = 'my_result'
iterations = 100
x = preprocess_image(target_image_path)
x = x.flatten()

In [16]:
for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x, fprime=evaluator.grads,maxfun=50)
    print('Current loss value:', min_val)
    img = x.copy().reshape((img_height, img_width, 3))
    img = deprocess_image(img)
    fname = result_prefix + '_at_iteration_%d.png' % i
    imageio.imwrite(fname, img)
    print('Image saved as', fname)
    end_time = time.time()
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 2514685200.0
Image saved as my_result_at_iteration_0.png
Iteration 0 completed in 20s
Start of iteration 1
Current loss value: 1213164300.0
Image saved as my_result_at_iteration_1.png
Iteration 1 completed in 14s
Start of iteration 2
Current loss value: 896076900.0
Image saved as my_result_at_iteration_2.png
Iteration 2 completed in 14s
Start of iteration 3
Current loss value: 740267460.0
Image saved as my_result_at_iteration_3.png
Iteration 3 completed in 14s
Start of iteration 4
Current loss value: 649760500.0
Image saved as my_result_at_iteration_4.png
Iteration 4 completed in 14s
Start of iteration 5
Current loss value: 592431000.0
Image saved as my_result_at_iteration_5.png
Iteration 5 completed in 14s
Start of iteration 6
Current loss value: 549639700.0
Image saved as my_result_at_iteration_6.png
Iteration 6 completed in 14s
Start of iteration 7
Current loss value: 517848640.0
Image saved as my_result_at_iteration_7.png
Iteration 7 complet

Current loss value: 304750140.0
Image saved as my_result_at_iteration_64.png
Iteration 64 completed in 14s
Start of iteration 65
Current loss value: 304202800.0
Image saved as my_result_at_iteration_65.png
Iteration 65 completed in 14s
Start of iteration 66
Current loss value: 303656320.0
Image saved as my_result_at_iteration_66.png
Iteration 66 completed in 14s
Start of iteration 67
Current loss value: 303094300.0
Image saved as my_result_at_iteration_67.png
Iteration 67 completed in 14s
Start of iteration 68
Current loss value: 302543580.0
Image saved as my_result_at_iteration_68.png
Iteration 68 completed in 14s
Start of iteration 69
Current loss value: 302053380.0
Image saved as my_result_at_iteration_69.png
Iteration 69 completed in 14s
Start of iteration 70
Current loss value: 301552930.0
Image saved as my_result_at_iteration_70.png
Iteration 70 completed in 14s
Start of iteration 71
Current loss value: 301079870.0
Image saved as my_result_at_iteration_71.png
Iteration 71 complet

KeyboardInterrupt: 